In [ ]:
# ==== WFO / Helpers (NO-WRITE) ====
import numpy as np, pandas as pd, yfinance as yf

# ---------- Data ----------
def download_data(ticker: str, lookback_years: int = 7) -> pd.DataFrame:
    """Download OHLCV daily data using yfinance. Returns tz-naive DatetimeIndex."""
    start = pd.Timestamp.today().normalize() - pd.DateOffset(years=lookback_years)
    df = yf.download(ticker, start=start.strftime("%Y-%m-%d"), auto_adjust=False, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.rename(columns={c: c.title() for c in df.columns})
    # keep only required columns
    need = ["Open","High","Low","Close","Volume"]
    for c in need:
        if c not in df:
            df[c] = np.nan
    df = df[need].copy()
    df.index = pd.to_datetime(df.index).tz_localize(None)
    df = df.dropna(subset=["Close"])
    return df

# ---------- Indicators ----------
def compute_atr(df: pd.DataFrame, period: int = 14) -> pd.Series:
    high, low, close = df["High"], df["Low"], df["Close"]
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low),
        (high - prev_close).abs(),
        (low  - prev_close).abs()
    ], axis=1).max(axis=1)
    atr = tr.rolling(period, min_periods=period).mean()
    return atr

def compute_sma_signals(df: pd.DataFrame, short: int, long_: int) -> pd.Series:
    """Binary raw long-only signal: 1 if SMA(short)>SMA(long), else 0."""
    s = df["Close"].rolling(short, min_periods=short).mean()
    l = df["Close"].rolling(long_, min_periods=long_).mean()
    raw = (s > l).astype(int)
    raw = raw.reindex(df.index).fillna(0)
    return raw

def apply_overlays(df: pd.DataFrame, raw: pd.Series,
                   entry_buf: int, exit_buf: int,
                   atr_mult: float, atr_period: int) -> pd.Series:
    """
    Debounce entries/exits with buffers; trailing stop using ATR.
    Returns position series (0/1). No writing to disk.
    """
    raw = raw.astype(int).reindex(df.index).fillna(0)
    atr = compute_atr(df, atr_period)
    close = df["Close"]
    pos = pd.Series(0, index=df.index, dtype=int)

    in_pos = 0
    cin = 0   # consecutive raw==1 while out of position
    cout = 0  # consecutive raw==0 while in position
    maxp = np.nan  # highest close since entry (for trailing stop)

    for i, (dt, cl) in enumerate(close.items()):
        r = int(raw.iloc[i])

        if in_pos == 0:
            cin = cin + 1 if r == 1 else 0
            if cin >= max(1, entry_buf):
                in_pos = 1
                maxp = cl
                cout = 0
        else:
            # update trailing max
            if np.isnan(maxp) or cl > maxp:
                maxp = cl
            # trailing stop check
            if not np.isnan(atr.iloc[i]):
                ts = maxp - atr_mult * atr.iloc[i]
                if cl <= ts:
                    in_pos = 0
                    cin = 0
                    maxp = np.nan
            # exit buffer on raw flip
            if in_pos == 1:
                cout = (cout + 1) if (r == 0) else 0
                if cout >= max(1, exit_buf):
                    in_pos = 0
                    cin = 0
                    maxp = np.nan

        pos.iloc[i] = in_pos

    return pos

# ---------- Backtest & Stats ----------
def vector_backtest(df: pd.DataFrame, pos: pd.Series, cost_bp: float = 0.0) -> dict:
    """
    Simple long-only backtest on Close-to-Close returns.
    pos is position (0/1). Trades at next bar open is ignored for simplicity.
    """
    pos = pos.reindex(df.index).fillna(0).astype(int)
    ret = df["Close"].pct_change().fillna(0.0)
    # apply position with 1-bar lag to avoid lookahead
    eff_pos = pos.shift(1).fillna(0)
    strat_ret = eff_pos * ret

    # transaction cost (bps) on position changes
    if cost_bp and cost_bp > 0:
        trades = eff_pos.diff().abs().fillna(0)  # 1 on enter, 1 on exit
        cost = trades * (cost_bp / 10000.0)
        strat_ret = strat_ret - cost

    equity = (1.0 + strat_ret).cumprod()
    return {"NetRet": strat_ret, "Equity": equity, "Position": eff_pos}

def perf_stats(equity: pd.Series, netret: pd.Series) -> dict:
    """Return Sharpe (ann 252), CAGR, MDD."""
    netret = netret.replace([np.inf, -np.inf], np.nan).dropna()
    equity = equity.replace([np.inf, -np.inf], np.nan).dropna()

    ann_factor = np.sqrt(252.0)
    mu = float(netret.mean())
    sd = float(netret.std(ddof=0))
    sharpe = (mu / sd * ann_factor) if sd > 0 else 0.0

    n = len(equity)
    if n == 0 or equity.iloc[0] <= 0:
        cagr = 0.0
    else:
        years = n / 252.0
        cagr = float(equity.iloc[-1]) ** (1.0 / years) - 1.0 if years > 0 else 0.0

    roll_max = equity.cummax()
    drawdown = equity / roll_max - 1.0
    mdd = float(drawdown.min()) if not drawdown.empty else 0.0

    return {"Sharpe": sharpe, "CAGR": cagr, "MDD": mdd}

# ---------- Optimizers ----------
def optimize_sma(df_tr: pd.DataFrame, df_te: pd.DataFrame,
                 short_grid=range(5, 31, 2), long_grid=range(40, 121, 5)) -> dict:
    """
    Grid-search short/long SMA on train, evaluate on test by Sharpe.
    Returns dict with best 'short','long'.
    """
    best = {"short": 10, "long": 50, "score": -np.inf}
    for sh in short_grid:
        for lg in long_grid:
            if sh >= lg: 
                continue
            raw_te = compute_sma_signals(df_te, sh, lg)
            bt_te = vector_backtest(df_te, raw_te)
            st = perf_stats(bt_te["Equity"], bt_te["NetRet"])
            score = st["Sharpe"]
            if np.isfinite(score) and score > best["score"]:
                best = {"short": sh, "long": lg, "score": float(score)}
    return best

def optimize_overlays(df_tr: pd.DataFrame, df_te: pd.DataFrame, sh: int, lg: int,
                      entry_buf_grid=(1,2,3), exit_buf_grid=(1,2,3),
                      atr_period_grid=(14, 21), atr_mult_grid=(2.0, 2.5, 3.0)) -> dict:
    """
    Grid-search simple overlay parameters on test by Sharpe.
    """
    best = {"entry_buf": 1, "exit_buf": 1, "atr_period": 14, "atr_mult": 2.0, "score": -np.inf}
    raw_te = compute_sma_signals(df_te, sh, lg)
    for eb in entry_buf_grid:
        for xb in exit_buf_grid:
            for ap in atr_period_grid:
                for am in atr_mult_grid:
                    pos_te = apply_overlays(df_te, raw_te, eb, xb, am, ap)
                    bt_te = vector_backtest(df_te, pos_te)
                    st = perf_stats(bt_te["Equity"], bt_te["NetRet"])
                    score = st["Sharpe"]
                    if np.isfinite(score) and score > best["score"]:
                        best = {"entry_buf": eb, "exit_buf": xb, "atr_period": ap, "atr_mult": float(am), "score": float(score)}
    return best


In [ ]:
# ==== MASTER BATCH: WFO for 25 tickers (NO WRITE, NO HEATMAP) ====
import json, numpy as np, pandas as pd

# ---------- ตรวจว่ามี helpers จากเซลล์ก่อนหน้าหรือยัง ----------
_need = ["download_data","optimize_sma","optimize_overlays",
         "compute_sma_signals","apply_overlays","vector_backtest","perf_stats"]
_missing = [n for n in _need if n not in globals()]
if _missing:
    raise RuntimeError(f"โปรดรันเซลล์ WFO/Helpers ก่อน: ขาดฟังก์ชัน {', '.join(_missing)}")

# ---------- Config ----------
LOOKBACK_YEARS = globals().get("LOOKBACK_YEARS", 7)
TRAIN_YEARS    = 3
TEST_MONTHS    = 6
STEP_MONTHS    = 6

# รายชื่อ 25 ตัว (ใช้ลิสต์ที่มี หรือกำหนดค่าเริ่มต้น)
SET25_TICKERS = globals().get("SET25_TICKERS", [
    "SCB.BK","SCC.BK","KBANK.BK","BBL.BK","KTB.BK",
    "CPALL.BK","CPN.BK","PTT.BK","PTTEP.BK","PTTGC.BK",
    "GULF.BK","ADVANC.BK","AOT.BK","BDMS.BK","BH.BK",
    "DELTA.BK","TRUE.BK","EA.BK","IVL.BK","MINT.BK",
    "BGRIM.BK","EGCO.BK","TOP.BK","OSP.BK","OR.BK",
])

def _make_windows(idx, train_years=3, test_months=6, step_months=6):
    """สร้างหน้าต่างเวลา (start_train → end_train | test → end_test) แบบ rolling"""
    start = idx.min(); end = idx.max()
    windows=[]
    cur_end_test = start + pd.DateOffset(years=train_years, months=test_months)
    while cur_end_test <= end:
        end_train = cur_end_test - pd.DateOffset(months=test_months)
        start_train = end_train - pd.DateOffset(years=train_years)
        if start_train < start: start_train = start
        windows.append((start_train.normalize(), end_train.normalize(), cur_end_test.normalize()))
        cur_end_test += pd.DateOffset(months=step_months)
    return windows

# ---------- รันสำหรับทุก ticker ----------
summary_rows = []
ARTIFACTS = {}  # เก็บผลต่อ-ticker ไว้ในหน่วยความจำ (ไม่มี heatmap)

for tk in SET25_TICKERS:
    print(f"\n==== {tk} ====")
    try:
        df = download_data(tk, LOOKBACK_YEARS)
    except Exception as e:
        print(f"[SKIP] {tk}: {e}")
        summary_rows.append({"ticker": tk, "windows": 0, "mean_Sharpe": np.nan,
                             "%Sharpe>0": np.nan, "mean_CAGR": np.nan, "mean_MDD": np.nan,
                             "last_short": None, "last_long": None,
                             "last_entry_buf": None, "last_exit_buf": None,
                             "last_atr_period": None, "last_atr_mult": None,
                             "note": f"download error: {e}"})
        ARTIFACTS[tk] = {"error": str(e)}
        continue

    windows = _make_windows(df.index, TRAIN_YEARS, TEST_MONTHS, STEP_MONTHS)
    if not windows:
        print(f"[SKIP] {tk}: windows not enough")
        summary_rows.append({"ticker": tk, "windows": 0, "mean_Sharpe": np.nan,
                             "%Sharpe>0": np.nan, "mean_CAGR": np.nan, "mean_MDD": np.nan,
                             "last_short": None, "last_long": None,
                             "last_entry_buf": None, "last_exit_buf": None,
                             "last_atr_period": None, "last_atr_mult": None,
                             "note": "no WFO windows"})
        ARTIFACTS[tk] = {"note": "no WFO windows"}
        continue

    rows=[]
    for (s_tr, e_tr, e_te) in windows:
        df_tr = df[(df.index>=s_tr) & (df.index<=e_tr)]
        df_te = df[(df.index>e_tr) & (df.index<=e_te)]
        if len(df_tr)<120 or len(df_te)<40:
            continue
        # 1) optimize SMA
        best_sma = optimize_sma(df_tr, df_te)
        sh, lg   = int(best_sma["short"]), int(best_sma["long"])
        # 2) optimize overlays
        best_ov  = optimize_overlays(df_tr, df_te, sh, lg)
        ebuf, xbuf = int(best_ov["entry_buf"]), int(best_ov["exit_buf"])
        ap, am     = int(best_ov["atr_period"]), float(best_ov["atr_mult"])
        # 3) OOS performance
        raw_te = compute_sma_signals(df_te, sh, lg)
        pos_te = apply_overlays(df_te, raw_te, ebuf, xbuf, am, ap)
        bt_te  = vector_backtest(df_te, pos_te)
        st_te  = perf_stats(bt_te["Equity"], bt_te["NetRet"])
        rows.append({
            "train": f"{s_tr.date()}→{e_tr.date()}",
            "test":  f"{(e_tr+pd.Timedelta(days=1)).date()}→{e_te.date()}",
            "short": sh, "long": lg,
            "entry_buf": ebuf, "exit_buf": xbuf,
            "atr_period": ap, "atr_mult": am,
            "test_Sharpe": round(st_te["Sharpe"],3),
            "test_CAGR": round(st_te["CAGR"],3),
            "test_MDD": round(st_te["MDD"],3),
        })

    wfo_df = pd.DataFrame(rows)
    if wfo_df.empty:
        print(f"[WARN] {tk}: no effective WFO rows")
        summary_rows.append({"ticker": tk, "windows": 0, "mean_Sharpe": np.nan,
                             "%Sharpe>0": np.nan, "mean_CAGR": np.nan, "mean_MDD": np.nan,
                             "last_short": None, "last_long": None,
                             "last_entry_buf": None, "last_exit_buf": None,
                             "last_atr_period": None, "last_atr_mult": None,
                             "note": "no effective WFO rows"})
        ARTIFACTS[tk] = {"note": "no effective WFO rows"}
        continue

    # summary metrics
    mean_sharpe = float(wfo_df["test_Sharpe"].mean())
    pct_pos     = float((wfo_df["test_Sharpe"]>0).mean())
    mean_cagr   = float(wfo_df["test_CAGR"].mean())
    mean_mdd    = float(wfo_df["test_MDD"].mean())
    last = wfo_df.iloc[-1]  # ใช้พารามิเตอร์จากหน้าต่างล่าสุดเป็นตัวแทน

    # สร้างสรุป (ในหน่วยความจำ) — ไม่มี heatmap
    summary = {
        "ticker": tk,
        "windows": int(len(wfo_df)),
        "mean_test_Sharpe": mean_sharpe,
        "pct_test_Sharpe_pos": pct_pos,
        "mean_test_CAGR": mean_cagr,
        "mean_test_MDD": mean_mdd,
        "last_params": {
            "short": int(last["short"]), "long": int(last["long"]),
            "entry_buf": int(last["entry_buf"]), "exit_buf": int(last["exit_buf"]),
            "atr_period": int(last["atr_period"]), "atr_mult": float(last["atr_mult"])
        }
    }

    # เก็บ artifacts ไว้ในหน่วยความจำ (ไม่มี heatmap)
    ARTIFACTS[tk] = {
        "wfo_df": wfo_df,
        "summary": summary
    }

    # ---- collect for overall table ----
    summary_rows.append({
        "ticker": tk,
        "windows": int(len(wfo_df)),
        "mean_Sharpe": round(mean_sharpe, 3),
        "%Sharpe>0": f"{pct_pos:.0%}",
        "mean_CAGR": round(mean_cagr, 3),
        "mean_MDD": round(mean_mdd, 3),
        "last_short": int(last["short"]),
        "last_long": int(last["long"]),
        "last_entry_buf": int(last["entry_buf"]),
        "last_exit_buf": int(last["exit_buf"]),
        "last_atr_period": int(last["atr_period"]),
        "last_atr_mult": float(last["atr_mult"]),
    })

# ---------- แสดงตารางสรุปคร่าว ๆ (ไม่เซฟไฟล์) ----------
SUMMARY_TABLE = pd.DataFrame(summary_rows)
if SUMMARY_TABLE.empty:
    print("ไม่มีผลลัพธ์ (อาจดาวน์โหลดข้อมูลไม่ได้หรือช่วงเวลาน้อยเกินไป)")
else:
    SUMMARY_TABLE = SUMMARY_TABLE.sort_values(["mean_Sharpe","mean_CAGR"], ascending=[False,False])
    cols = ["ticker","windows","mean_Sharpe","%Sharpe>0","mean_CAGR","mean_MDD",
            "last_short","last_long","last_entry_buf","last_exit_buf","last_atr_period","last_atr_mult"]
    display(SUMMARY_TABLE[cols])
    print("\nหมายเหตุ: ผลลัพธ์ทั้งหมดอยู่ในตัวแปร ARTIFACTS และ SUMMARY_TABLE (ไม่มีการเขียนไฟล์/ไม่มี heatmap)")


In [ ]:
# ==== PLOT TOP-N TICKERS BY WFO RESULT (NO WRITE; uses SUMMARY_TABLE / ARTIFACTS) ====
import json, numpy as np, pandas as pd, yfinance as yf
import plotly.graph_objects as go
from IPython.display import display, Markdown

# ---------------- Config ----------------
N_PLOTS = 6                    # จำนวนกราฟสูงสุดที่จะพล็อต
SELECT_MODE = "top_sharpe"     # 'top_sharpe' | 'recent_signal'
LAST_N_DAYS = 7                # ใช้เมื่อ SELECT_MODE='recent_signal'
LOOKBACK_YEARS = globals().get("LOOKBACK_YEARS", 7)

# Initial chart view
INIT_X_RANGE = "1y"            # '1y'|'6m'|'3m'|'all'
LOCK_Y_ON_INIT = True          # ล็อกสเกล Y ครั้งแรก (กัน outlier), แล้วค่อยซูม/แพนได้
Y_RANGE_MODE = "quantile"      # 'quantile' 1%–99% | 'full'

# ---------------- Guards ----------------
_need = ["download_data","apply_overlays","compute_atr"]
_missing = [n for n in _need if n not in globals()]
if _missing:
    raise RuntimeError(f"โปรดรันเซลล์ helpers ก่อน: ขาดฟังก์ชัน {', '.join(_missing)}")

if "ARTIFACTS" not in globals() and "SUMMARY_TABLE" not in globals():
    raise RuntimeError("ไม่พบ ARTIFACTS/SUMMARY_TABLE ในหน่วยความจำ: โปรดรัน MASTER BATCH (no-write) ก่อน")

# ---------------- Utilities ----------------
def _xrange(idx, mode="1y"):
    if len(idx) == 0: return None
    if mode == "all": return None
    n = 252 if mode=="1y" else (126 if mode=="6m" else (63 if mode=="3m" else len(idx)))
    i0 = max(0, len(idx)-n)
    return [idx[i0], idx[-1]]

def _init_y_range(series_list, mode="quantile"):
    stack = pd.concat(series_list, axis=1).astype(float)
    arr = stack.values.ravel()
    arr = arr[np.isfinite(arr)]
    if arr.size == 0: return None
    if mode == "quantile":
        lo = float(np.nanquantile(arr, 0.01)); hi = float(np.nanquantile(arr, 0.99))
    else:
        lo = float(np.nanmin(arr)); hi = float(np.nanmax(arr))
    pad = max(1.0, abs(hi)*0.05) if (not np.isfinite(hi-lo) or (hi-lo)==0) else 0.07*(hi-lo)
    return [lo - pad, hi + pad]

def _last_event(df, short, long_, ebuf, xbuf, atr_p, atr_m):
    s = df["Close"].rolling(short, min_periods=short).mean()
    l = df["Close"].rolling(long_,  min_periods=long_).mean()
    raw = (s>l).astype(int)
    pos = apply_overlays(df, raw, ebuf, xbuf, atr_m, atr_p)
    ch  = pos.diff().fillna(0)
    trades = (ch>0).astype(int) + (ch<0).astype(int)*-1
    ev = df[trades!=0].copy()
    ev["Side"] = np.where(trades[trades!=0]==1,"BUY","SELL")
    # trailing stop series (เพื่อแสดง)
    atr = compute_atr(df, atr_p)
    ts = pd.Series(np.nan, index=df.index, dtype=float)
    p=0; cin=0; cout=0; maxp=np.nan
    for i, (dt,row) in enumerate(df.iterrows()):
        raw_i = 1 if s.iloc[i]>l.iloc[i] else 0
        if p==0:
            cin = cin+1 if raw_i==1 else 0
            if cin>=max(1,ebuf): p=1; maxp=row["Close"]; cout=0
        else:
            if np.isnan(maxp) or row["Close"]>maxp: maxp=row["Close"]
            if not np.isnan(atr.iloc[i]):
                ts.iloc[i] = maxp - atr_m*atr.iloc[i]
                if row["Close"] <= ts.iloc[i]: p=0; cin=0; maxp=np.nan
            if p==1:
                if raw_i==0: cout+=1
                else: cout=0
                if cout>=max(1,xbuf): p=0; cin=0; maxp=np.nan
    return ev, trades, s, l, pos, ts

def plot_signals_plotly_from_params(ticker, params, lookback_years=LOOKBACK_YEARS):
    # โหลดข้อมูล
    df = download_data(ticker, lookback_years)
    ev, trades, s, l, pos, ts = _last_event(df, params["short"], params["long"],
                                            params["entry_buf"], params["exit_buf"],
                                            params["atr_period"], params["atr_mult"])
    buys  = df[trades==1] if trades is not None else df.iloc[0:0]
    sells = df[trades==-1] if trades is not None else df.iloc[0:0]

    # เตรียมกราฟ
    y_init = _init_y_range([df["Close"], s, l, ts], Y_RANGE_MODE)
    x_init = _xrange(df.index, INIT_X_RANGE)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df["Close"], name="Close", mode="lines"))
    fig.add_trace(go.Scatter(x=df.index, y=s, name=f"SMA({params['short']})", mode="lines"))
    fig.add_trace(go.Scatter(x=df.index, y=l, name=f"SMA({params['long']})",  mode="lines"))
    fig.add_trace(go.Scatter(x=df.index, y=ts, name=f"TS {params['atr_mult']}×ATR{params['atr_period']}",
                             mode="lines", line=dict(dash="dash")))
    fig.add_trace(go.Scatter(x=buys.index, y=buys["Close"], name="BUY",
                             mode="markers", marker=dict(symbol="triangle-up", size=12,
                             color="green", line=dict(width=1, color="darkgreen"))))
    fig.add_trace(go.Scatter(x=sells.index, y=sells["Close"], name="SELL",
                             mode="markers", marker=dict(symbol="triangle-down", size=12,
                             color="red", line=dict(width=1, color="darkred"))))
    fig.update_layout(
        template="plotly_white",
        title=f"{ticker} — Signals (SMA + Overlays)",
        hovermode="x unified", legend=dict(orientation="h"),
        xaxis=dict(rangeslider=dict(visible=False),
                   rangeselector=dict(buttons=[
                       dict(count=1,label="1m",step="month",stepmode="backward"),
                       dict(count=3,label="3m",step="month",stepmode="backward"),
                       dict(count=6,label="6m",step="month",stepmode="backward"),
                       dict(count=1,label="YTD",step="year",stepmode="todate"),
                       dict(count=1,label="1y",step="year",stepmode="backward"),
                       dict(count=3,label="3y",step="year",stepmode="backward"),
                       dict(step="all")
                   ]),
                   range=x_init),
        yaxis=dict(autorange=(not LOCK_Y_ON_INIT),
                   range=(y_init if LOCK_Y_ON_INIT and y_init is not None else None)),
        yaxis_title="Price (THB)"
    )
    fig.show(config={"scrollZoom": True, "displayModeBar": True})

    # สรุปสัญญาณล่าสุด
    last_sig_date = None
    last_sig_side = ""
    if ev is not None and not ev.empty:
        last_sig_date = ev.index.max()
        last_sig_side = ev.loc[last_sig_date, "Side"]
    status_now = "LONG" if (pos.iloc[-1]==1) else "CASH"
    return {
        "last_signal_date": str(last_sig_date.date()) if last_sig_date is not None else "",
        "last_signal": last_sig_side,
        "status_now": status_now
    }

# ---------------- Build plotting list (from memory, no files) ----------------
def _get_summary_df_in_memory():
    if "SUMMARY_TABLE" in globals() and isinstance(SUMMARY_TABLE, pd.DataFrame) and not SUMMARY_TABLE.empty:
        return SUMMARY_TABLE.copy()
    # ถ้าไม่มี SUMMARY_TABLE ให้ลองสร้างจาก ARTIFACTS
    if "ARTIFACTS" in globals() and isinstance(ARTIFACTS, dict) and ARTIFACTS:
        rows = []
        for tk, art in ARTIFACTS.items():
            summ = art.get("summary") or {}
            lp = (summ.get("last_params") or {})
            rows.append({
                "ticker": summ.get("ticker", tk),
                "windows": summ.get("windows", np.nan),
                "mean_Sharpe": summ.get("mean_test_Sharpe", np.nan),
                "%Sharpe>0": summ.get("pct_test_Sharpe_pos", np.nan),
                "mean_CAGR": summ.get("mean_test_CAGR", np.nan),
                "mean_MDD": summ.get("mean_test_MDD", np.nan),
                "last_short": lp.get("short"),
                "last_long": lp.get("long"),
                "last_entry_buf": lp.get("entry_buf"),
                "last_exit_buf": lp.get("exit_buf"),
                "last_atr_period": lp.get("atr_period"),
                "last_atr_mult": lp.get("atr_mult"),
            })
        return pd.DataFrame(rows)
    return pd.DataFrame()

summary_df = _get_summary_df_in_memory()
if summary_df.empty:
    raise RuntimeError("ไม่พบผลลัพธ์ WFO ในหน่วยความจำ: โปรดรัน MASTER BATCH (no-write) ก่อน")

# เลือก tickers ที่จะพล็อต
tickers_to_plot = []
if SELECT_MODE == "top_sharpe":
    picks = summary_df.sort_values(["mean_Sharpe", "mean_CAGR"], ascending=[False, False]).head(N_PLOTS)
    tickers_to_plot = picks["ticker"].tolist()
elif SELECT_MODE == "recent_signal":
    cutoff = pd.Timestamp.today().normalize() - pd.Timedelta(days=LAST_N_DAYS)
    for _, r in summary_df.iterrows():
        tk = r["ticker"]
        # ดึง params ล่าสุดจาก ARTIFACTS/summary
        params = None
        if "ARTIFACTS" in globals() and tk in ARTIFACTS and ARTIFACTS[tk].get("summary"):
            params = ARTIFACTS[tk]["summary"].get("last_params", None)
        if not params:
            # ลองจากคอลัมน์ใน summary_df เอง (กรณีไม่มี ARTIFACTS)
            keys = ["last_short","last_long","last_entry_buf","last_exit_buf","last_atr_period","last_atr_mult"]
            if all(k in r and pd.notna(r[k]) for k in keys):
                params = {
                    "short": int(r["last_short"]), "long": int(r["last_long"]),
                    "entry_buf": int(r["last_entry_buf"]), "exit_buf": int(r["last_exit_buf"]),
                    "atr_period": int(r["last_atr_period"]), "atr_mult": float(r["last_atr_mult"]),
                }
        if not params:
            continue

        df = download_data(tk, LOOKBACK_YEARS)
        ev, *_ = _last_event(df, int(params["short"]), int(params["long"]),
                             int(params["entry_buf"]), int(params["exit_buf"]),
                             int(params["atr_period"]), float(params["atr_mult"]))
        if ev is not None and not ev.empty and ev.index.max() >= cutoff:
            tickers_to_plot.append(tk)
    tickers_to_plot = tickers_to_plot[:N_PLOTS]
else:
    tickers_to_plot = summary_df["ticker"].tolist()[:N_PLOTS]

display(Markdown(f"### กราฟสัญญาณ (โหมด: **{SELECT_MODE}**), จำนวน {len(tickers_to_plot)} ตัว"))
if not tickers_to_plot:
    display(Markdown("> ไม่มีตัวที่ตรงเงื่อนไขให้พล็อต"))
else:
    # สรุปหน้าจอแบบสั้น ๆ ก่อนพล็อต
    preview_rows = []
    for tk in tickers_to_plot:
        # params ล่าสุดจาก ARTIFACTS ถ้ามี; ไม่งั้นจาก summary_df
        params = None
        if "ARTIFACTS" in globals() and tk in ARTIFACTS and ARTIFACTS[tk].get("summary"):
            params = ARTIFACTS[tk]["summary"]["last_params"]
        if not params:
            r = summary_df.loc[summary_df["ticker"]==tk].iloc[0]
            params = {
                "short": int(r["last_short"]), "long": int(r["last_long"]),
                "entry_buf": int(r["last_entry_buf"]), "exit_buf": int(r["last_exit_buf"]),
                "atr_period": int(r["last_atr_period"]), "atr_mult": float(r["last_atr_mult"]),
            }

        info = plot_signals_plotly_from_params(tk, params, lookback_years=LOOKBACK_YEARS)
        preview_rows.append({
            "ticker": tk,
            "SMA": f"{params['short']}/{params['long']}",
            "Overlays": f"EB{params['entry_buf']}/XB{params['exit_buf']}, ATR{params['atr_period']}×{params['atr_mult']}",
            "status_now": info["status_now"],
            "last_signal_date": info["last_signal_date"],
            "last_signal": info["last_signal"]
        })
    disp = pd.DataFrame(preview_rows)
    display(disp[["ticker","status_now","last_signal_date","last_signal","SMA","Overlays"]])


In [ ]:
import os, datetime
print(f"{os.environ.get('RUN_MARKER','รันเสร็จแล้ว:')} {datetime.datetime.now()}")
